In [ ]:
%load_ext autoreload
%autoreload 2

from loguru import logger
from typing import List

from mistral_fine_tuning.segment import extract_jokes_from_segments, Segment
from mistral_fine_tuning.utils import extract_routines_ids

In [ ]:
def extract_repertories(
        segments: List[Segment],
    ):
    repertoires = extract_jokes_from_segments(segments = segments)
    return repertoires

In [ ]:
routine_ids = extract_routines_ids(folder='../data/raw/transcripts')

for routine_id in routine_ids:
    logger.info(f"Processing routine {routine_id}")

    filename = f"../data/raw/transcripts/routine_{routine_id}_transcript.jsonl"
    segments = []

    with open(filename, "r", encoding="utf-8") as file:
        for line in file:
            segment = Segment.from_json(line)
            segments.append(segment)
    repertoires = extract_repertories(segments = segments)

    for r in repertoires:
        for joke in r.jokes:
            try:
                if len(joke.corrected_transcript) > 0:
                    with open(f"../data/interim/jokes/routine_{routine_id}_repertoire.jsonl", "a", encoding="utf-8") as file:
                        json_line = joke.json()
                        file.write(json_line + "\n")
            except Exception as e:
                logger.info(f"Error writing joke for {routine_id} {e}")
                continue